In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pk
#import networkx as nx
import re 

path = "d:/Facultad/Tesis/"
path_git = "d:/Git_Proyects/Tesis/Base_Twitter/Tweets_August/"
base_nac = pd.read_csv(path+'Corpus_medios_nac.csv')
Tweets_mayo = pd.read_csv(path_git+'Tweets_August_solo_medios.csv')

C:\Users\Ale\AppData\Local\Temp\ipykernel_1260\555256001.py:10: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  base_nac = pd.read_csv(path+'Corpus_medios_nac.csv')


In [2]:
def replace(x):
    x = str(x)
    x = x.replace('”', '"')
    x = x.replace('“', '"')
    x = x.replace('”', '"')
    x = x.replace('\x93', '"')
    x = x.replace('\x94', '"')
    
    return x

def extract_quotes(x):
    return re.findall('"([^"]*)"', x)

def words_length(x):
    return len(x.split(" "))

def pre_process_quoted(df1):
    df = df1.copy()
#    df['nota'] = df['nota'].apply(replace)
    df.loc[:, 'nota'] = df['nota'].apply(replace)
    #df['citas']  = df['nota'].apply(extract_quotes)
    df.loc[:, 'citas'] = df['nota'].apply(extract_quotes)

    df_quoutes2 = pd.DataFrame({'Fecha': df['fecha'].iloc[1:], 'Hora': df['hora'].iloc[1:], 'Cita': df['citas'].iloc[1:], 'Link': df['link'].iloc[1:], 'Nota': df['nota'].iloc[1:]})
    df_quoutes = df_quoutes2.explode('Cita') # Cada cita sea una fila distinta
    df_quoutes = df_quoutes.dropna(subset=['Cita']) # Elimino la filas que no haya quotes

    #df_quoutes['Citas'] = df_quoutes['Citas'].apply(replace)

    df_quoutes = df_quoutes.reset_index()
    df_quoutes = df_quoutes.drop(['index'], axis = 1)
    df_quoutes['Cant_Palabras'] = df_quoutes['Cita'].apply(words_length) # genero columna que cuente la cantidad de plabaras
    df_quoutes_pf = df_quoutes[df_quoutes['Cant_Palabras'] >= 4]
    df_quoutes_pf = df_quoutes_pf[df_quoutes_pf['Cant_Palabras'] < 60]
    df_quoutes_pf = df_quoutes_pf.reset_index()
    df_quoutes_pf = df_quoutes_pf.drop(['index'], axis = 1)
    return df_quoutes_pf

In [5]:
Tweets_mayo[Tweets_mayo['link_nacionales'] == 1]

,Unnamed: 0,id,Tweets,Tw_limpios,Fecha,urls,medios,source,link_nacionales
5,106,1156899491657723905,"2)\n\nEl pasado 25 de julio de 2019, unos 150 ...","2 ) pasado 25 julio 2019 , 150 artistas , acad...",Thu Aug 01 12:08:27 +0000 2019,https://tn.com.ar/politica/unas-150-personalid...,False,Twitter Web App,1
8,296,1156961688182300672,DEL CUAL SE SIRVIÓ MUCHOS AÑOS. https://t.co/...,SIRVIÓ AÑOS .,Thu Aug 01 16:15:36 +0000 2019,https://www.xn--lamaanaonline-lkb.com.ar/notic...,False,Twitter Web Client,1
12,842,1157106742355988482,@lufrann Lo dice todo aquel que entiende la di...,lufrann dice aquel entiende diferencia votar t...,Fri Aug 02 01:52:00 +0000 2019,https://www.lanacion.com.ar/politica/miguel-pi...,False,Twitter for Android,1
14,1150,1157273389431701505,Habla el OraCulo Asis\n\nhttps://t.co/dNHu8iNCvX,Habla OraCulo Asis,Fri Aug 02 12:54:11 +0000 2019,https://www.lanacion.com.ar/politica/jorge-asi...,False,Twitter for Android,1
15,1154,1157274946323828736,El Turco ayer se zarpó con la caña con ruda\nh...,Turco ayer zarpó caña ruda,Fri Aug 02 13:00:22 +0000 2019,https://www.lanacion.com.ar/politica/jorge-asi...,False,Twitter Web App,1
...,...,...,...,...,...,...,...,...,...
37761,7006071,1167598732969484288,El país que vota Ladrones.\n#CostaPobre\n@alfe...,país vota Ladrones . alferdezprensa alferdez C...,Sat Aug 31 00:43:25 +0000 2019,https://www.clarin.com/politica/juicio-cristin...,False,Twitter Web App,1
37762,7012773,1167945471429492736,@latanaguerrera @DanceSalmon @Lowuelaharket @j...,latanaguerrera DanceSalmon Lowuelaharket jmkar...,Sat Aug 31 23:41:14 +0000 2019,https://www.infobae.com/politica/2019/08/14/es...,False,Twitter Web App,1
37763,7014109,1167972218036457472,Les agradeceriamos prueben su inocencia caso c...,agradeceriamos prueben inocencia caso contrari...,Sun Sep 01 01:27:31 +0000 2019,https://www.lanacion.com.ar/politica/dos-perit...,False,Twitter Web App,1
37764,7014186,1167974252999405574,UD @CFKArgentina psicopata de m... mechera la...,UD CFKArgentina psicopata m ... mechera mujer ...,Sun Sep 01 01:35:36 +0000 2019,https://www.lanacion.com.ar/politica/la-plata-...,False,Twitter for Android,1


In [7]:
base_nac['medios_comp'] = base_nac['link'].isin(Tweets_mayo['urls'])
base_compartida = base_nac[base_nac['medios_comp']==True]
base_compartida

,link,medio,provincia,seccion,fecha,hora,autor,volanta,titulo,subtitulo,nota,tags,medios_comp
16353,https://www.clarin.com/politica/maria-eugenia-...,clarin,Buenos Aires,politica,2019/02/17,00:30,none,Elecciones 2019,"María Eugenia Vidal, en campaña: ya visitó dos...",La gobernadora planea reunirse con los intende...,La gobernadora María Eugenia Vidal completó ...,"['María Eugenia Vidal', 'Elecciones 2019']",True
16393,https://www.clarin.com/politica/jose-luis-espe...,clarin,Buenos Aires,politica,2019/02/18,18:45,none,La pelea por el 2019,"José Luis Espert dio el sí, se va de luna de m...",El economista se casó con su mujer y se fue a...,El economista devenido candidato decidió pasa...,"['Jose Luis Espert', 'Elecciones 2019']",True
16428,https://www.clarin.com/politica/ex-contador-cr...,clarin,Buenos Aires,politica,2019/02/20,00:45,lucía salinas,Los cuadernos de las coimas,El ex contador de Cristina Kirchner dio detall...,Víctor Manzanares volvió a declarar ante el ...,"Por tercera vez, y durante varias horas, Víct...","['Los Cuadernos De Las Coimas', 'Víctor Manza...",True
16598,https://www.clarin.com/politica/favor-hagan-pe...,clarin,Buenos Aires,politica,2019/02/26,16:00,none,Durante un acto,"“Por favor, hagan algo”, el pedido de un obrer...",El Presidente recorrió obras en la Ciudad de ...,"Sin convocatoria a la prensa, Mauricio Macri r...","['Mauricio Macri', 'Horacio Rodríguez Larreta']",True
16649,https://www.clarin.com/politica/espert-dice-pe...,clarin,Buenos Aires,politica,2019/02/27,22:00,paula lugones,Gira por EE.UU.,Espert dice que habrá que pedir más dinero al FMI,"“Un poquito más del Fondo vamos a necesitar”,...",El economista José Luis Espert llegó a Washi...,"['Jose Luis Espert', 'Fondo Monetario Internac...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
469730,http://www.losprimeros.tv/nota/118861/se-defin...,losprimerostv,Tucuman,politica,2019/08/29,NaN,fuente: infobae,,Se definió cuándo declarará CFK en el juici...,,El Tribunal Oral Federal 2 definió el orden e...,"['Cristina', 'juicio', 'Kirchner']",True
469731,http://www.losprimeros.tv/nota/118858/alberto-...,losprimerostv,Tucuman,politica,2019/08/29,NaN,fuente: infobae,,Alberto Fernández a Macri: “Debe estar contan...,,En un breve encuentro con la prensa luego de r...,"['elecciones', 'Fernández', 'Macri']",True
469741,http://www.losprimeros.tv/nota/119064/alberto-...,losprimerostv,Tucuman,politica,2019/08/30,NaN,NaN,,"Alberto Fernández, ante The Wall Street Journ...",,"El candidato a presidente del Frente de Todos,...","['Alberto Fernández', 'Default', 'economía']",True
469747,http://www.losprimeros.tv/nota/118985/lavagna-...,losprimerostv,Tucuman,politica,2019/08/30,NaN,fuente: diario popular,,Lavagna propuso un Programa de Emergencia Alim...,,"Roberto Lavagna, el candidato presidencial por...","['Alimentaria', 'emergencia', 'Lavagna', 'Prog...",True


In [8]:
extract_quotes  = pre_process_quoted(base_compartida)
extract_quotes

,Fecha,Hora,Cita,Link,Nota,Cant_Palabras
0,2019/02/18,18:45,"Si bien ya son familia, ahora vuelven a elegir...",https://www.clarin.com/politica/jose-luis-espe...,El economista devenido candidato decidió pasa...,19
1,2019/02/18,18:45,cortar con la decadencia argentina y sacar el ...,https://www.clarin.com/politica/jose-luis-espe...,El economista devenido candidato decidió pasa...,10
2,2019/02/18,18:45,Es una gira política que yo la catalogaría d...,https://www.clarin.com/politica/jose-luis-espe...,El economista devenido candidato decidió pasa...,12
3,2019/02/18,18:45,"con los principales bancos de inversión, en W...",https://www.clarin.com/politica/jose-luis-espe...,El economista devenido candidato decidió pasa...,41
4,2019/02/20,00:45,Bochi era el López Rega de Néstor: siempre l...,https://www.clarin.com/politica/ex-contador-cr...,"Por tercera vez, y durante varias horas, Víct...",14
...,...,...,...,...,...,...
18001,2019/08/30,NaN,o de,http://www.losprimeros.tv/nota/118985/lavagna-...,"Roberto Lavagna, el candidato presidencial por...",4
18002,2019/08/31,NaN,el resultado de las políticas que llevó a ca...,http://www.losprimeros.tv/nota/119292/cristina...,La candidata a vicepresidenta por el Frente de...,13
18003,2019/08/31,NaN,Sabíamos que íbamos a ganar pero no por esta...,http://www.losprimeros.tv/nota/119292/cristina...,La candidata a vicepresidenta por el Frente de...,10
18004,2019/08/31,NaN,El resultado de las PASO es el resultado de la...,http://www.losprimeros.tv/nota/119292/cristina...,La candidata a vicepresidenta por el Frente de...,34


In [9]:
citas = extract_quotes['Cita']
for i in citas:
    print(i)

Si bien ya son familia, ahora vuelven a elegirse, frente al Estado y toda su familia que los acompaña
cortar con la decadencia argentina y sacar el país adelante
Es una gira política que yo la catalogaría de trato prácticamente presidencial
con los principales bancos de inversión, en Washington nos reuniremos con los principales 'think tanks' liberales, tenemos una agenda de reuniones con miembros del gobierno americano, el Congreso americano, con autoridades del Fondo Monetario Internacional y en Miami con importantes inversores
Bochi era el López Rega de Néstor: siempre le decía que todo estaba bien
circuito económico basado en la actividad hotelera
a maniobras de blanqueo
fondos líquidos para poder justificar la adquisición de esos establecimientos hoteleros
Cristina  está haciendo una construcción en Los Sauces y necesita plata blanca
Ahí se arreglaron los 105.000 dólares por el edificio pelado por mes y Relats se hizo cargo del gerenciamiento
recibir ese dinero y a

In [10]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')  
phrases_emb = model.encode(citas, convert_to_tensor=True) 

C:\Users\Ale\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
pk.dump(phrases_emb, open(path_git+'emb_august.pk', 'wb'))
#pk.load(open(path_git+'emb_august.pk', 'rb'))

In [11]:
clusters = util.community_detection(phrases_emb,threshold=0.8, min_community_size = 1)  
cluste_map = {citas[j]: str(i+1) for i, clus in enumerate(clusters) for j in clus}
extract_quotes['id_cluster'] = extract_quotes['Cita'].map(cluste_map)

In [46]:
for cit in extract_quotes[extract_quotes['id_cluster'] == '10']['Cita']:
    print(cit)

La Argentina no tiene posibilidad de caer en default si yo soy presidente
la Argentina no tiene ninguna posibilidad de caer en default
Argentina no tiene ninguna posibilidad de caer en default si yo soy presidente
La Argentina no tiene ninguna posibilidad de caer en default si soy presidente
La Argentina no tiene ninguna posibilidad de caer en default si soy presidente
Argentina no tiene ninguna posibilidad de caer en default si soy presidente
no hay ninguna chance de que Argentina caiga en default
Argentina no tiene ninguna posibilidad de caer en default si yo soy presidente
La Argentina no tiene ninguna posibilidad de caer en default si soy presidente, eso sería muy dañino, nadie puede plantear una quita. Este fantasma alentado por el Gobierno es un error
la Argentina no tiene ninguna posibilidad de caer en default
La Argentina no tiene ninguna posibilidad de caer en default
la Argentina no tiene ninguna posibilidad de caer en default
La Argentina no tiene ninguna posibilidad de ca

In [47]:
muchas_citas = []
for i in range(1, 200):
    citas = []
    for cit in extract_quotes[extract_quotes['id_cluster'] == str(i)]['Cita']:
        citas.append(cit)
        if len(cit) > len(citas[0]):
            citas[0] = cit
    muchas_citas.append(citas[0])

In [48]:
file = open(path_git+'muchas_citas_august.txt', 'w', encoding='utf8')
for cita in muchas_citas:
    file.write(cita+'\n')
file.close()

In [11]:
len(clusters)

5877

In [ ]:
pk.dump(clusters,open(path+'oct_Tw/'+nombre+'clusters_75_oct.pk', 'wb'))

TOKEN = "6287446315:AAFAnvbB6vUSzttp-smI5E00jDP7hNI7kCo"
chat_id = "6045013691"
message = f"Termine de correr el codigo que clusteriza con 0.85 con medios nacionales con data del 07al 12  del 2021"
url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"                               
print(requests.get(url).json())

In [1]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')  

C:\Users\Ale\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [67]:
import numpy as np
from numpy.linalg import norm  
def cosine_similarity(A, B):
    return np.dot(A, B) / (norm(A) * norm(B))

frase1 = 'física'
frase2 = 'mecánica'

f1 = model.encode(frase1)
f2 = model.encode(frase2)

cosine_similarity(f1,f2)
len(f1)

768